In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import gsw

from scipy.integrate import solve_ivp

In [2]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [3]:
lat = 35+8.4585/60
bini = 15

Smooth ADCP time series
* 30-min running median to remove spikes
* 60-min running mean to smooth

In [4]:
ds['Eas_sm'] = ds['Eas'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()
ds['Nor_sm'] = ds['Nor'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()

In [5]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_sm'][:,bini]);
plt.plot(ds['time'],ds['Eas'][:,bini],'--');
plt.plot(ds['time'],ds['Eas_sm'][:,1],'-');
#plt.plot(ds['time'],ds['Eas'][:,1]);
plt.ylabel('m/s')
plt.title('eastward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'eastward velocity')

In [6]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_sm'][:,bini]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

### Test with limited obs

#### Next steps:
* Iterate to find forcing F (check for consistency in force balance at top bin, adjust F if necessary)
* Logarithmic grid spacing

In [7]:
zf = ds['binheight'][bini] # upper boundary height
zobs = np.array(ds['binheight'][0:(bini+1)])

In [80]:
zo = 0.03 # roughness length
i = 1j
kappa = 0.41 # Von Karman constant
f = gsw.f(lat)

z = np.logspace(np.log10(zo),np.log10(zf),201)
dz = np.diff(z)

In [81]:
gi = np.isfinite(ds['Eas_sm'][:,0])
tobs = np.array((ds['time'][gi] - ds['time'][gi][0])/np.timedelta64(1,'s')) # array of times in seconds
deltat = tobs[1]-tobs[0]

In [82]:
wobs = np.array(ds['Eas_sm'][gi,0:(bini+1)]+i*ds['Nor_sm'][gi,0:(bini+1)])

w0_obs = np.interp(z,zobs[1:bini],wobs[0,1:bini])[1:-1]
w0 = np.concatenate([np.real(w0_obs),
                           np.imag(w0_obs)])

dwobsdt = np.gradient(wobs,deltat,axis=0)

In [83]:
def dwdt_bbl(t, w_in):
    # Note that w_in is real and complex parts concatenates
    # need to combine into complex numbers
    # Scipy docs say that ode solvers RK45 and BDF can be used in complex plane
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.BDF.html#scipy.integrate.BDF
    # but perhaps this does not work for *arrays* of complex numbers?
    # Test this and check Numpy Github issues for similar problems
    
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = np.interp(t,tobs,wobs[:,bini])
    dwinfdt = np.interp(t,tobs,dwobsdt[:,bini])

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [95]:
#%%timeit

#tf = tobs[-1]
tf = 86400*30

ntimes = len()

tic = np.datetime64('now')
print(tic)
a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, t_eval = tobs[tobs<=tf], method='BDF')
toc = np.datetime64('now')
print(toc)
print(toc-tic)
#a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, method='BDF')

2020-01-07T00:36:26
2020-01-07T00:48:40
-734 seconds


In [85]:
N = int(np.shape(a.y)[0]/2)

t_sol = a.t
w_sol = a.y[:N,:]+i*a.y[N:,:]

In [86]:
plt.figure()
plt.subplot(211)
plt.plot(t_sol,np.real(w_sol).T,color='gray')
plt.plot(tobs[0:12],np.real(wobs[0:12,bini]))

plt.subplot(212)
plt.plot(t_sol,np.imag(w_sol).T,color='gray')
plt.plot(tobs[0:12],np.imag(wobs[0:12,bini]))

FigureCanvasNbAgg()

In [87]:
np.diff(a.t[0:100])

array([300., 300., 300., 300., 300., 300., 300., 300., 300., 300., 300.,
       300.])

### Test with steady forcing

In [88]:
winf_steady = 0.04+i*0
w0_steady = np.concatenate([np.real(winf_steady)*np.ones(N),
                           np.imag(winf_steady)*np.ones(N)])
zo = 0.01

def dwdt_bbl_steady(t, w_in):
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = winf_steady
    dwinfdt = 0+i*0

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    #print(dwdt)
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [89]:
modsteady = solve_ivp(dwdt_bbl_steady, t_span=[0,2*86400], 
                      y0 = w0_steady, method='BDF')

In [90]:
t_sol_steady = modsteady.t
w_sol_steady = modsteady.y[:N,:]+i*modsteady.y[N:,:]

In [91]:
plt.figure()
plt.subplot(121)
plt.plot(np.real(w_sol_steady[:,-1]),z[1:-1]);
plt.plot(np.imag(w_sol_steady[:,-1]),z[1:-1]);
#plt.plot(tobs[0:12*24],wobs[0:12*24,bini])

FigureCanvasNbAgg()

In [92]:
ustar_steady = kappa*zo*np.abs(w_sol_steady[1,-1])/dz[0]

In [93]:
ustar_steady

0.0021174047257213075

In [94]:
dz

array([0.00064156, 0.00065528, 0.00066929, 0.00068361, 0.00069823,
       0.00071316, 0.00072841, 0.00074399, 0.0007599 , 0.00077615,
       0.00079275, 0.0008097 , 0.00082702, 0.0008447 , 0.00086277,
       0.00088122, 0.00090006, 0.00091931, 0.00093897, 0.00095905,
       0.00097956, 0.00100051, 0.0010219 , 0.00104376, 0.00106608,
       0.00108888, 0.00111216, 0.00113595, 0.00116024, 0.00118505,
       0.00121039, 0.00123628, 0.00126272, 0.00128972, 0.0013173 ,
       0.00134547, 0.00137425, 0.00140364, 0.00143365, 0.00146431,
       0.00149563, 0.00152761, 0.00156028, 0.00159365, 0.00162773,
       0.00166254, 0.00169809, 0.00173441, 0.0017715 , 0.00180938,
       0.00184807, 0.0018876 , 0.00192796, 0.00196919, 0.00201131,
       0.00205432, 0.00209825, 0.00214312, 0.00218895, 0.00223577,
       0.00228358, 0.00233241, 0.00238229, 0.00243324, 0.00248527,
       0.00253842, 0.00259271, 0.00264815, 0.00270479, 0.00276263,
       0.00282171, 0.00288205, 0.00294369, 0.00300664, 0.00307